In [1]:
import pandas as pd
df = pd.read_pickle('yonhap_ready.pkl')

In [2]:
df.head(3)

,article,title,sec1,sec_idx
0,'회장이 청부폭력' 루머 협박범도 기소…사이버명예훼손 전담수사 첫 사례 ...,"""해경이 가만있으라 방송"" 허위사실 유포 40대 기소",사회,4
1,지난 해 9월 북한의 함경북도 항구도시 나진과 러시아 극동지역 도시 ...,南 점검단 방북…나진-하산 프로젝트 시범운송 시작(종합),정치,6
2,포항 도착한 화물선 러시아 시베리아산 유연탄 4만 500t을...,나진-하산 프로젝트 탄력…시범수송 석탄 포항 도착(종합),경제,0


In [3]:
X = df.article
y = df.sec_idx
print(X.shape)
print(y.shape)

(458081,)
(458081,)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(343560,)
(114521,)
(343560,)
(114521,)


In [5]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 60.9 ms


### Summary:
    * vect.fit(train) learns the vocabulary of the training data
    * vect.transform(train) uses the fitted vocabulary to build a document term matrix from the training data
    * vect.transform(test) uses the fitted vocabulary to build a document term matrix from the testing data(and ignores tokens it hasn't seen before)

In [6]:
stopwords_news=['지난해', '중','말', '뒤', '곳', '군', '위', '개', '간', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [7]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_news, ngram_range=(1,3), min_df=2, max_df=.5)
X_train_dtm = vect.fit_transform(X_train)

CPU times: user 19min 26s, sys: 15.5 s, total: 19min 42s
Wall time: 19min 42s


In [8]:
X_test_dtm = vect.transform(X_test)

In [9]:
X_train_dtm.shape

(343560, 9371220)

In [10]:
X_test_dtm.shape

(114521, 9371220)

## save vector & dtm

https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python
https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn

In [11]:
import pickle

with open("yonhap_tfidf_vect.pkl", 'wb') as handle:
    pickle.dump(vect, handle)

with open("yonhap_dtm.pkl", 'wb') as handle:
    pickle.dump(X_test_dtm, handle)

In [12]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)

In [13]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 5.59 s, sys: 496 ms, total: 6.09 s
Wall time: 6.09 s


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [14]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [15]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.85785139843347513

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.91      0.89      0.90     29038
         과학       0.61      0.54      0.57       645
         국제       0.85      0.89      0.87     22269
         문화       0.75      0.91      0.82      6086
         사회       0.83      0.84      0.84     29780
         생활       0.54      0.74      0.62      2183
         정치       0.92      0.82      0.87     24520

avg / total       0.86      0.86      0.86    114521



In practice Multinomial NB can be better than Linear SVM in some situations and svm better than NB on other datasets. Apparently t's possible to combine the two approaches to get a very good baseline: projects:nbsvm - Sida I. Wang

You can also try non-linear SVMs but often the quadratic complexity of the SMO algorithm (for instance as implemented in libsvm) makes it not practical on datasets with more than 5000 documents. Instead on prefer to use liblinear than can only train linear SVM on large datasets.

https://www.quora.com/What-are-the-differences-similarities-between-SVM-Naive-Bayes-for-binary-text-classification-wrt-how-they-are-processing-the-features

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [18]:
from sklearn import svm
#clf = svm.SVC(decision_function_shape='ovo')
clfrSVM=svm.LinearSVC(C=1)

In [19]:
%time clfrSVM.fit(X_train_dtm, y_train)

CPU times: user 2min, sys: 740 ms, total: 2min
Wall time: 2min


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [20]:
y_pred_class_svm = clfrSVM.predict(X_test_dtm)

In [21]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_svm)

0.89598414264632686

In [22]:
print(classification_report(y_test, y_pred_class_svm, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.90      0.94      0.92     29038
         과학       0.73      0.58      0.65       645
         국제       0.89      0.91      0.90     22269
         문화       0.88      0.87      0.87      6086
         사회       0.89      0.90      0.90     29780
         생활       0.79      0.60      0.68      2183
         정치       0.92      0.87      0.89     24520

avg / total       0.90      0.90      0.90    114521



https://stackoverflow.com/questions/40115043/no-space-left-on-device-error-while-fitting-sklearn-model

In [23]:
import numpy as np
counts = np.bincount(y_pred_class[(y_pred_class!=2) & (y_test==2)])
print(np.argmax(counts))

0


In [24]:
X_test[(y_pred_class==0) & (y_test==2)]

340928           지난달 캐나다 일자리가 3만5천개 늘어 11개월 연속 증가 행진을 기록...
186273             보스턴컨설팅그룹 "내년 아태지역이 두 번째로 부유해져" 전망    ...
16170            상파울루 증권사 직원       브라질 중앙은행이 기준금리를 연 14....
4584             베트남 하노이에 있는 한 휴대전화 매장. 삼성전자와 애플 등의 각종 모...
239631           오쓰카가구의 창업자인 오쓰카 가쓰히사(勝久·왼쪽) 전 회장과 그의 딸인...
115583           14일 베이징에서 리진자오 중국 국가여유국장과의 회담을 마친 뒤 한국 ...
308586            애플이 20일(현지시간) 세계 최대 모바일 칩 메이커인 퀄컴에 10억...
315779           철강업체 고베(神戶)제강의 품질 조작과 닛산(日産)자동차의 무자격 직원...
445302           11일 연속 절상되던 위안화가 다시 약세로 돌아선 가운데 중국 당국이 ...
38023            아마리 아키라 일본 경제재생담당상(AFP.연합뉴스.자료사진)  두 분기...
39151            증권사 객장에서 생각에 빠진 중국 투자자       중국 증시 강세의 ...
390413                            최신 유행 트렌드 총집결 #흥(클릭!)    
161753           세계최대 중장비업체 '캐터필러' 주가 7.7% 폭등   '인프라 재건'...
402750           독일 최대 은행인 도이체방크에 중국 자본이 손을 뻗기 시작했다.  독일...
2635             베트남의 한 의류공장       베트남 의류산업이 급성장하며 세계적인 ...
427180           산업소비재 중심으로 사업구조를 재편 중인 미국의 제너럴일렉트릭(GE)이...
349881                               최신 

In [26]:
df[df.index==421591]

,article,title,sec1,sec_idx
421591,최신 유행 트렌드 총집결 #흥(클릭!),"코미 ""FBI, 러시아 대선개입…트럼프팀과 공모 수사중""(속보)",국제,2


In [25]:
with open("yonhap_svm_clf.pkl", 'wb') as handle:
    pickle.dump(clfrSVM, handle)

In [26]:
with open("yonhap_nb_clf.pkl", 'wb') as handle:
    pickle.dump(nb, handle)